# Análise Exploratória de Dados

In [1]:
import pandas as pd
import numpy as np
from numpy import mean
from numpy import std
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.preprocessing import label_binarize, LabelEncoder, MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc, classification_report, ConfusionMatrixDisplay
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestRegressor
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import joblib
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [8]:
df = pd.read_csv("PT_16P - 16P.csv", encoding='ISO-8859-1')
df

,ID da Resposta,VocÃª regularmente faz novos amigos,VocÃª passa muito do seu tempo livre explorando vÃ¡rios tÃ³picos aleatÃ³rios que despertam seu interesse,Ver outras pessoas chorando pode facilmente fazer vocÃª sentir vontade de chorar tambÃ©m,VocÃª frequentemente faz um plano de backup para um plano de backup,"VocÃª geralmente permanece calmo, mesmo sob muita pressÃ£o","Em eventos sociais, vocÃª raramente tenta se apresentar a novas pessoas e geralmente fala com aquelas que jÃ¡ conhece",VocÃª prefere terminar completamente um projeto antes de comeÃ§ar outro,VocÃª Ã© muito sentimental,VocÃª gosta de usar ferramentas de organizaÃ§Ã£o como horÃ¡rios e listas,...,VocÃª acredita que ponderar sobre questÃµes filosÃ³ficas abstratas Ã© uma perda de tempo,VocÃª se sente mais atraÃ­do por lugares com atmosferas movimentadas e agitadas do que por lugares quietos e Ã­ntimos,VocÃª sabe Ã primeira vista como alguÃ©m estÃ¡ se sentindo,VocÃª frequentemente se sente sobrecarregado,VocÃª completa as coisas metodicamente sem pular nenhuma etapa,VocÃª Ã© muito intrigado por coisas rotuladas como controversas,VocÃª passaria uma boa oportunidade adiante se achasse que outra pessoa precisava mais dela,VocÃª tem dificuldades com prazos,VocÃª se sente confiante de que as coisas vÃ£o dar certo para vocÃª,Personalidade
0,0,0,0,0,0,0,1,1,0,0,...,0,0,0,-1,0,0,0,0,0,ENFP
1,1,0,0,-2,-3,-1,2,-2,0,3,...,0,-2,0,2,0,-1,-1,-1,3,ISFP
2,2,0,0,2,0,-1,2,0,0,1,...,0,2,0,2,-1,0,1,2,1,INFJ
3,3,0,-1,3,-1,0,0,-2,0,-2,...,0,0,-1,-1,0,1,0,-2,-1,ISTP
4,4,0,0,-1,0,2,-1,-2,0,1,...,0,1,0,2,0,1,-1,2,-1,ENFJ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59994,59994,0,-1,0,-3,-2,2,1,0,1,...,0,2,0,1,0,0,2,3,3,INFJ
59995,59995,0,0,-2,1,3,0,-3,0,2,...,0,1,0,0,0,0,2,0,-2,ESTP
59996,59996,0,0,1,0,-2,-2,0,0,-1,...,-1,2,0,-3,0,1,-1,0,-1,ISTP
59997,59997,0,0,1,-1,-2,-1,2,0,1,...,0,2,0,2,0,0,0,1,0,ISTJ


In [13]:

mapa = {'ESFJ': 0, 'ESFP': 1, 'ESTJ': 2, 'ESTP': 3, 'ENFJ': 4, 'ENFP': 5,
         'ENTJ': 6, 'ENTP': 7, 'ISFJ': 8, 'ISFP': 9, 'ISTJ': 10, 'ISTP': 11,
           'INFJ': 12, 'INFP': 13, 'INTJ': 14, 'INTP': 15}
df['Personalidade'] = df['Personalidade'].replace(mapa)
colunas_por_indice = df.drop(df.columns[[27, 55, 23, 51, 56, 9, 45, 1, 7, 31, 47, 22, 50, 33, 49, 0, 52, 20]], axis=1)

colunas_por_indice.to_csv('quest.csv', index=False)

In [1]:
# Fazer leitura do .csv
df = pd.read_csv("data/16P.csv", encoding='ISO-8859-1')

NameError: name 'pd' is not defined

In [8]:
df = pd.read_csv("quest.csv", encoding='ISO-8859-1')
df.columns = range(df.shape[1])

# Save the DataFrame to a CSV file
df.to_csv('quest_new.csv', index=False)

In [ ]:
df.info()

Após fazer uma verificação inicial, as colunas do dataframe estão todas no formato int64. Exceto a coluna dos tipos das personalidades.

In [ ]:
# Verificar a quantidade de personalidades
df['Personality'].value_counts()

In [ ]:
df['Personality'].value_counts().plot(kind='bar')

In [ ]:
# Verificar a quantidade de valores nulos
df.isnull().sum()

In [ ]:
# Mapeando as personalidades


# Mapenado os nomes das colunas em Portugues 
mapa_portugues = {'You regularly make new friends.': 'Faz novos amigos',
                  'You spend a lot of your free time exploring various random topics that pique your interest': 'Passa muito tempo explorando topicos que te interessam',
                  'Seeing other people cry can easily make you feel like you want to cry too': 'Ver outras pessoas chorando faz você querer chorar',
                  'You often make a backup plan for a backup plan.':'Você costuma fazer planos de backup', 
                  'You usually stay calm, even under a lot of pressure': 'Você costuma ficar calmo mesmo sob pressão',
                  'At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know': 'Em eventos sociais, você raramente tenta se apresentar para novas pessoas e fala principalmente com as que você já conhece',
                  'You prefer to completely finish one project before starting another.': 'Você prefere terminar completamente um projeto antes de começar outro',
                  'You are very sentimental.': 'Você é muito sentimental',
                  'You like to use organizing tools like schedules and lists.': 'Você gosta de usar ferramentas de organização como agendas e listas',
                  'Even a small mistake can cause you to doubt your overall abilities and knowledge.': 'Mesmo um pequeno erro pode fazer você duvidar de suas habilidades e conhecimentos',
                  'You feel comfortable just walking up to someone you find interesting and striking up a conversation.': 'Você se sente confortável apenas indo até alguém que você acha interessante e começando uma conversa',
                  'You are not too interested in discussing various interpretations and analyses of creative works.': 'Você não está muito interessado em discutir várias interpretações e análises de trabalhos criativos',
                  'You are more inclined to follow your head than your heart.': 'Você é mais inclinado a seguir sua cabeça do que seu coração',
                  'You usually prefer just doing what you feel like at any given moment instead of planning a particular daily routine.': 'Você geralmente prefere fazer o que sente em qualquer momento em vez de planejar uma rotina diária específica',
                  'You rarely worry about whether you make a good impression on people you meet.': 'Você raramente se preocupa se causa uma boa impressão nas pessoas que conhece',
                  'You enjoy participating in group activities.': 'Você gosta de participar de atividades em grupo',
                  'You like books and movies that make you come up with your own interpretation of the ending.': 'Você gosta de livros e filmes que fazem você criar sua própria interpretação do final',
                  'Your happiness comes more from helping others accomplish things than your own accomplishments.': 'Sua felicidade vem mais de ajudar os outros a realizar coisas do que de suas próprias realizações',
                  'You are interested in so many things that you find it difficult to choose what to try next.': 'Você está interessado em tantas coisas que acha difícil escolher o que tentar a seguir',
                  'You are prone to worrying that things will take a turn for the worse.': 'Você tem tendência a se preocupar que as coisas vão piorar',
                  'You avoid leadership roles in group settings.': 'Você evita papéis de liderança em configurações de grupo',
                  'You are definitely not an artistic type of person.': 'Você definitivamente não é um tipo de pessoa artística',
                  'You think the world would be a better place if people relied more on rationality and less on their feelings.': 'Você acha que o mundo seria um lugar melhor se as pessoas confiassem mais na racionalidade e menos em seus sentimentos',
                  'You prefer to do your chores before allowing yourself to relax.': 'Você prefere fazer suas tarefas antes de se permitir relaxar',
                  'You enjoy watching people argue.': 'Você gosta de assistir as pessoas discutirem',
                  'You tend to avoid drawing attention to yourself.': 'Você tende a evitar chamar a atenção para si mesmo',
                  'Your mood can change very quickly.': 'Seu humor pode mudar muito rapidamente',
                  'You lose patience with people who are not as efficient as you.': 'Você perde a paciência com pessoas que não são tão eficientes quanto você',
                  'You often end up doing things at the last possible moment.': 'Você frequentemente acaba fazendo as coisas no último momento possível',
                  'You have always been fascinated by the question of what, if anything, happens after death.': 'Você sempre foi fascinado pela pergunta do que, se algo, acontece após a morte',
                  'You usually prefer to be around others rather than on your own.': 'Você geralmente prefere estar com os outros em vez de sozinho',
                  'You become bored or lose interest when the discussion gets highly theoretical.': 'Você fica entediado ou perde o interesse quando a discussão fica altamente teórica',
                  'You find it easy to empathize with a person who has gone through something you never have.': 'Você acha fácil se colocar no lugar de uma pessoa que passou por algo que você nunca passou',
                  'You usually postpone finalizing decisions for as long as possible.': 'Você geralmente adia a finalização de decisões o máximo possível',
                  'You rarely second-guess the choices that you have made.': 'Você raramente duvida das escolhas que fez',
                  'After a long and exhausting week, a lively social event is just what you need.': 'Depois de uma semana longa e exaustiva, um evento social animado é exatamente o que você precisa',
                  'You enjoy going to art museums.': 'Você gosta de ir a museus de arte',
                  'You often have a hard time understanding other people’s feelings.': 'Você frequentemente tem dificuldade em entender os sentimentos de outras pessoas',
                  'You like to have a to-do list for each day.': 'Você gosta de ter uma lista de tarefas para cada dia',
                  'You rarely feel insecure.': 'Você raramente se sente inseguro',
                  'You avoid making phone calls.': 'Você evita fazer ligações',
                  'You often spend a lot of time trying to understand views that are very different from your own.': 'Você frequentemente passa muito tempo tentando entender pontos de vista muito diferentes do seu',
                  'In your social circle, you are often the one who contacts your friends and initiates activities.': 'Em seu círculo social, você é frequentemente o que entra em contato com seus amigos e inicia atividades',
                  'If your plans are interrupted, your top priority is to get back on track as soon as possible.': 'Se seus planos são interrompidos, sua prioridade é voltar ao normal o mais rápido possível',
                  'You are still bothered by mistakes that you made a long time ago.': 'Você ainda está incomodado com erros que cometeu há muito tempo',
                  'You rarely contemplate the reasons for human existence or the meaning of life.': 'Você raramente contempla as razões da existência humana ou o significado da vida',
                  'Your emotions control you more than you control them.': 'Suas emoções te controlam mais do que você as controla',
                  'You take great care not to make people look bad, even when it is completely their fault.': 'Você toma muito cuidado para não fazer as pessoas parecerem más, mesmo quando é completamente culpa delas',
                  'Your personal work style is closer to spontaneous bursts of energy than to organized and consistent efforts.': 'Seu estilo de trabalho pessoal é mais próximo de explosões espontâneas de energia do que de esforços organizados e consistentes',
                  'When someone thinks highly of you, you wonder how long it will take them to feel disappointed in you.': 'Quando alguém pensa muito em você, você se pergunta quanto tempo levará até que eles fiquem desapontados com você',
                  'You would love a job that requires you to work alone most of the time.': 'Você adoraria um trabalho que exige que você trabalhe sozinho na maior parte do tempo',
                  'You believe that pondering abstract philosophical questions is a waste of time.': 'Você acredita que ponderar questões filosóficas abstratas é uma perda de tempo',
                  'You feel more drawn to places with busy, bustling atmospheres than quiet, intimate places.': 'Você se sente mais atraído por lugares com atmosferas movimentadas e agitadas do que lugares tranquilos e íntimos',
                  'You know at first glance how someone is feeling.': 'Você sabe à primeira vista como alguém está se sentindo',
                  'You often feel overwhelmed.': 'Você frequentemente se sente sobrecarregado',
                  'You complete things methodically without skipping over any steps.': 'Você completa as coisas metodicamente sem pular nenhuma etapa',
                  'You are very intrigued by things labeled as controversial.': 'Você é muito intrigado por coisas rotuladas como controversas',
                  'You would pass along a good opportunity if you thought someone else needed it more.': 'Você passaria uma boa oportunidade se achasse que outra pessoa precisava mais dela',
                  'You struggle with deadlines.': 'Você tem dificuldade com prazos',
                  'You feel confident that things will work out for you.': 'Você tem confiança de que as coisas vão dar certo para você'}

# Mapeando os nomes das colunas reduzidos em Inglês
mapa_reduzido = {'You regularly make new friends.': 'New frineds',
                  'You spend a lot of your free time exploring various random topics that pique your interest': 'Spend time exploring topics',
                  'Seeing other people cry can easily make you feel like you want to cry too': 'Seeing people cry',
                  'You often make a backup plan for a backup plan.':'Backup plan', 
                  'You usually stay calm, even under a lot of pressure': 'Stay calm',
                  'At social events, you rarely try to introduce yourself to new people and mostly talk to the ones you already know': 'Social events',
                  'You prefer to completely finish one project before starting another.': 'Finish project',
                  'You are very sentimental.': 'Sentimental',
                  'You like to use organizing tools like schedules and lists.': 'Organizing tools',
                  'Even a small mistake can cause you to doubt your overall abilities and knowledge.': 'Doubt abilities',
                  'You feel comfortable just walking up to someone you find interesting and striking up a conversation.': 'Walking up to someone',
                  'You are not too interested in discussing various interpretations and analyses of creative works.': 'Discussing creative works',
                  'You are more inclined to follow your head than your heart.': 'Follow head',
                  'You usually prefer just doing what you feel like at any given moment instead of planning a particular daily routine.': 'Doing what you feel',
                  'You rarely worry about whether you make a good impression on people you meet.': 'Good impression',
                  'You enjoy participating in group activities.': 'Group activities',
                  'You like books and movies that make you come up with your own interpretation of the ending.': 'Interpretation of the ending',
                  'Your happiness comes more from helping others accomplish things than your own accomplishments.': 'Helping others',
                  'You are interested in so many things that you find it difficult to choose what to try next.': 'Difficult to choose',
                  'You are prone to worrying that things will take a turn for the worse.': 'Worrying',
                  'You avoid leadership roles in group settings.': 'Avoid leadership',
                  'You are definitely not an artistic type of person.': 'Artistic',
                  'You think the world would be a better place if people relied more on rationality and less on their feelings.': 'Rationality',
                  'You prefer to do your chores before allowing yourself to relax.': 'Chores',
                  'You enjoy watching people argue.': 'Watching people argue',
                  'You tend to avoid drawing attention to yourself.': 'Avoid attention',
                  'Your mood can change very quickly.': 'Mood change',  
                  'You lose patience with people who are not as efficient as you.': 'Lose patience',
                  'You often end up doing things at the last possible moment.': 'Last possible moment',
                  'You have always been fascinated by the question of what, if anything, happens after death.': 'After death',
                  'You usually prefer to be around others rather than on your own.': 'Around others',
                  'You become bored or lose interest when the discussion gets highly theoretical.': 'Bored',
                  'You find it easy to empathize with a person whose experiences are very different from yours.': 'Empathize',
                  'You usually postpone finalizing decisions for as long as possible.': 'Postpone decisions',
                  'You rarely second-guess the choices that you have made.': 'Second-guess',
                  'After a long and exhausting week, a lively social event is just what you need.': 'Social event',
                  'You enjoy going to art museums.': 'Art museums',
                  'You often have a hard time understanding other people’s feelings.': 'Understanding feelings',
                  'You like to have a to-do list for each day.': 'To-do list',
                  'You rarely feel insecure.': 'Insecure',
                  'You avoid making phone calls.': 'Avoid calls',
                  'You often spend a lot of time trying to understand views that are very different from your own.': 'Understand views',
                  'In your social circle, you are often the one who contacts your friends and initiates activities.': 'Social circle',
                  'If your plans are interrupted, your top priority is to get back on track as soon as possible.': 'Back on track',
                  'You are still bothered by mistakes that you made a long time ago.': 'Bothered by mistakes',
                  'You rarely contemplate the reasons for human existence or the meaning of life.': 'Meaning of life',
                  'Your emotions control you more than you control them.': 'Emotions control',
                  'You take great care not to make people look bad, even when it is completely their fault.': 'Make people look bad',
                  'Your personal work style is closer to spontaneous bursts of energy than to organized and consistent efforts.': 'Work style',
                  'When someone thinks highly of you, you wonder how long it will take them to feel disappointed in you.': 'Disappointed',
                  'You would love a job that requires you to work alone most of the time.': 'Work alone',
                  'You believe that pondering abstract philosophical questions is a waste of time.': 'Philosophical questions',
                  'You feel more drawn to places with busy, bustling atmospheres than quiet, intimate places.': 'Bustling atmosphere',
                  'You know at first glance how someone is feeling.': 'First glance',
                  'You often feel overwhelmed.': 'Overwhelmed',
                  'You complete things methodically without skipping over any steps.': 'Complete things',
                  'You are very intrigued by things labeled as controversial.': 'Controversial',
                  'You would pass along a good opportunity if you thought someone else needed it more.': 'Pass along',
                  'You struggle with deadlines.': 'Deadlines',
                  'You feel confident that things will work out for you.': 'Confident'}

In [ ]:
# Dropar a coluna 'Response Id'
df.drop('Response Id', axis=1, inplace= True)

# Substituir as personalidades pelos valores mapeados
df['Personality'] = df['Personality'].replace(mapa)

# Substituir os nomes das colunas pelos valores mapeados reduzidos
dfreduced = df.rename(columns=mapa_reduzido)

# Substituir os nomes das colunas pelos valores mapeados em português
dfport = df.rename(columns=mapa_portugues)



In [ ]:
df.head()

In [ ]:
dfreduced.head()

In [ ]:
dfport.head()

In [ ]:
df.describe()

In [ ]:
#Para trabalhar mais facilmente com as colunas, mudei o nome delas para o índice que elas estão
mapping = {col: i for i, col in enumerate(df.columns)}
df.rename(columns=mapping, inplace=True)

df.head()

In [ ]:
colunas_0_1 = df.columns[(df.min() == 0) & (df.max() == 1)]
colunas_nao_0_1 = df.columns[(df.min() != 0) | (df.max() != 1)]

print("Colunas com valores entre 0 e 1:")
print(colunas_0_1)

print("\nColunas com valores fora da faixa de 0 a 1:")
print(colunas_nao_0_1)

#scaler = MinMaxScaler()
#df[colunas_nao_0_1] = scaler.fit_transform(df[colunas_nao_0_1])

#df

In [ ]:
# Traçando um gráfico de correlação pra ver quais colunas tem maior correlação com outras.
corr_matrix = df.corr()

plt.figure(figsize=(20,20))
sns.heatmap(df.corr(), cmap='Blues', fmt='.2f', linewidths=2)
plt.title('Correlação entre as variáveis do dataset 16P')
plt.show()
#Eu sei que isso ta horrivel foi só pra testar mesmo

In [ ]:
print(corr_matrix)

In [ ]:
# vou ordenar pra facilitar a visualização
correlation_values = corr_matrix.unstack().sort_values(ascending=False)

pd.set_option('display.max_rows', None)

# Filtrar as correlações de uma coluna com ela propria
correlation_values = correlation_values[correlation_values.index.get_level_values(0) != correlation_values.index.get_level_values(1)]

# print(correlation_values)

In [ ]:
# Resetar o índice
correlation_values = correlation_values.reset_index()
correlation_values.columns = ['Variable1', 'Variable2', 'Correlation']

In [ ]:
# Definir as faixas de correlação
faixas = {
    'correlação baixa': (0.3, 0.5),
    'média': (0.5, 0.7),
    'alta': (0.7, 0.99),
    'correlação baixa inversamente proporcional': (-0.5, -0.3),
    'média inversamente proporcional': (-0.7, -0.5),
    'alta inversamente proporcional': (-0.99, -0.7)
}

# Imprimir as correlações dentro de cada faixa
for faixa, (min_val, max_val) in faixas.items():
    correlacoes = corr_matrix[(corr_matrix >= min_val) & (corr_matrix < max_val)]
    if not correlacoes.empty:
        print(f'Faixa de correlação: {faixa}')
        for col1 in correlacoes.columns:
            for col2 in correlacoes.index:
                valor_correlacao = correlacoes.loc[col2, col1]
                if not pd.isna(valor_correlacao):
                    print(f'Variável {col1} e Variável {col2}: {valor_correlacao}')
        print('\n')

In [ ]:
correlation_matrix_2 = df.corr()
correlation_with_target = correlation_matrix_2.iloc[:-1, -1]
plt.figure(figsize=(60, 20))
sns.heatmap(correlation_with_target.to_frame(), cmap='coolwarm', annot=True, fmt=".2f", cbar=True, square=True)
plt.title('Correlação entre Features e Target')
plt.xlabel('Features')
plt.ylabel('Target')
plt.show()

In [ ]:
print(correlation_with_target)

In [ ]:
# Criar dicionário para armazenar as features em cada faixa
features_por_faixa = {faixa: [] for faixa in faixas}

# Iterar sobre a série de correlação
for feature, correlacao in correlation_with_target.items():
    for faixa, (minimo, maximo) in faixas.items():
        if minimo <= correlacao < maximo:
            features_por_faixa[faixa].append(feature)

# Exibir as features em cada faixa
for faixa, features in features_por_faixa.items():
    print(f'Faixa de correlação: {faixa}')
    print(f'Features: {features}')
    print('\n')


In [ ]:
# Plotando histogramas para cada personalidade
for i in range(20):
    fig, axs = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle('Opção escolhida pelo usuário para a pergunta: ')
    for j in range(3):
        axs[j].hist(df.iloc[:, i*3 + j].dropna())
        axs[j].set_title(j + i*3) #Como os títulos sao muito grandes, decidi mudar para o numero da coluna

plt.show()

In [ ]:
# Após fazer a análise da correlação e observar os histogramas, podemos ver que algumas variáveis não possuem uma correlação 
# muito alta com a variável target.
# Para isso, será interessante remover as variáveis que possuem uma correlação menor que 0.01 e maior que -0.01 com o target.

dfcorrelation = df.drop(correlation_with_target[(correlation_with_target < 0.01) & (correlation_with_target > -0.01)].index, axis=1)

In [ ]:
dfcorrelation.head()

# Regressão Logística

### Definindo o target

In [ ]:
# Analisando o dataframe
df.head()

In [ ]:
# Separando as colunas entre X e y
X = df.iloc[:, 0:60]
y = df[60]

### Criando o Modelo

In [ ]:
# Define the multinomial logistic regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

### Treinando e Avaliando o Modelo

#### 1. Validação Cruzada

In [ ]:
# Define the model evaluation procedure
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)

# Acurácia em cada fold
accuracies = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)

# Plotar o gráfico
plt.figure(figsize=(10,5))
plt.plot(accuracies, label='Acurácia')
plt.xlabel('Fold', fontweight="bold")
plt.ylabel('Accuracy', fontweight="bold")
plt.title('Histórico de Acurácia Durante a Validação Cruzada', fontweight="bold")
plt.legend()
plt.show()

# Variação da acurácia ao longo das iterações da validação cruzada para visualizar a estabilidade e a confiabilidade do modelo

In [ ]:
# Acurácia média do treinamento com validação cruzada
print('Acurácia média: %.3f (%.3f)' % (mean(accuracies), std(accuracies)))

#### 2. Matriz de confusão

In [ ]:
# Dividir os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Treinar o modelo
model.fit(X_train, y_train)

# Fazer previsões
y_pred = model.predict(X_test)

# Criar a matriz de confusão
cm = confusion_matrix(y_test, y_pred)

# Plotar a matriz de confusão
plt.figure(figsize=(10,7))
sns.heatmap(cm, annot=True, fmt='g', cmap='Blues')
plt.xlabel('Predicted', fontsize=12, fontweight="bold")
plt.ylabel('True', fontsize=12, fontweight="bold")
plt.title('Matriz de Confusão', fontweight="bold", fontsize=14)
plt.show()

#### 3. Curva ROC

In [ ]:
# Binarizar as labels
y_bin = label_binarize(y, classes=np.unique(y))

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y_bin, test_size=0.25, random_state=42)

# Treinar o modelo com estratégia One-vs-Rest
ovr_model = OneVsRestClassifier(model)
ovr_model.fit(X_train, y_train)

# Calcular as probabilidades de cada classe
y_prob = ovr_model.predict_proba(X_test)

# Plotar a curva ROC para cada classe
plt.figure(figsize=(10,8))
for i in range(y_prob.shape[1]):
    fpr, tpr, _ = roc_curve(y_test[:, i], y_prob[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label='Class %d (area = %0.2f)' % (i, roc_auc))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate', fontweight="bold")
plt.ylabel('True Positive Rate', fontweight="bold")
plt.title('Curva ROC para cada classe', fontweight="bold")
plt.legend(loc="lower right")
plt.show()

# È uma representação gráfica da capacidade do modelo de discriminar entre classes

# Random Forest

Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

In [ ]:
y_pred2 = classifier.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred2)
print(cm)
accuracy_score(y_test, y_pred2)

In [ ]:
importances = classifier.feature_importances_
indices = np.argsort(importances)[::-1]
plt.figure(figsize=(10, 6))
plt.title("Importâncias das Características")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), X.columns[indices], rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
report = classification_report(y_test, y_pred2)
print("Classification Report:\n", report)

In [ ]:
"""
    Aqui, eu antes tinha utilizado uma documentação do github para aplicar a Random-Forest, mas a accuracy usando o scikit-learn diretamente foi maior,
então optei por manter o resultado final assim.
==================================================================
Dados anteriores:
    train_preds = rf_model.predict(X_set=X_train)
    print("TRAIN PERFORMANCE")
    print("Train size", len(Y_train))
    print("True preds", sum(train_preds == Y_train))
    print("Accuracy", sum(train_preds == Y_train) / len(Y_train))

    test_preds = rf_model.predict(X_set=X_test)
    print("TEST PERFORMANCE")
    print("Test size", len(Y_test))
    print("True preds", sum(test_preds == Y_test))
    print("Accuracy", sum(test_preds == Y_test) / len(Y_test))
    ============================================================
TRAIN PERFORMANCE:
    Train size 44999
    True preds 36516
    Accuracy 0.8114846996599925
    TEST PERFORMANCE
    Test size 15000
    True preds 12027
    Accuracy 0.8018

"""

Neural-Network


In [ ]:
df = pd.read_csv("data/16P.csv", encoding='ISO-8859-1')
dfx = pd.read_csv("data/16P.csv", encoding='ISO-8859-1')

In [ ]:
df.drop('Response Id', axis=1, inplace= True)
df

In [ ]:
#Para trabalhar mais facilmente com as colunas, mudei o nome delas para o índice que elas estão
# mapping = {col: i for i, col in enumerate(df.columns)}
# df.rename(columns=mapping, inplace=True)

In [ ]:
mapa = {'ESFJ': 0, 'ESFP': 1, 'ESTJ': 2, 'ESTP': 3, 'ENFJ': 4, 'ENFP': 5,
         'ENTJ': 6, 'ENTP': 7, 'ISFJ': 8, 'ISFP': 9, 'ISTJ': 10, 'ISTP': 11,
           'INFJ': 12, 'INFP': 13, 'INTJ': 14, 'INTP': 15}

# Substituir as personalidades pelos valores mapeados
df['Personality'] = df['Personality'].replace(mapa)

In [ ]:
df

In [ ]:
dfx.drop(columns=['Response Id','Personality'], axis=1, inplace= True)

In [ ]:
dfx

In [ ]:
# le = LabelEncoder()
# dff = le.fit_transform(dff)
# dff

In [ ]:
X = torch.tensor(dfx.values, dtype=torch.int64)
y = torch.tensor(df['Personality'].values, dtype=torch.long)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
loaders = {
    'train': DataLoader(
        list(zip(X_train, y_train)),
        batch_size=32,
        shuffle=True
    ),
    'test': DataLoader(
        list(zip(X_test, y_test)),
        batch_size=32,
        shuffle=True
    )
}

In [ ]:
class NeuralNet_a(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNet_a, self).__init__()
        self.linear_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Softmax()
        )

    def forward(self, x):
        return self.linear_stack(x)

In [ ]:
model = NeuralNet_a(16, 32, 1)
optimizer = optim.Adam(model.parameters(), lr=1e-2)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
train_loss_hist = []
train_acc_hist = []
test_loss_hist = []
test_acc_hist = []

epochs = 200

progress = tqdm(range(epochs*len(loaders['train'])))

for epoch in range(1, epochs+1):
    epoch_loss = []
    epoch_acc = []

    model.train()
    for data, target in loaders['train']:
        optimizer.zero_grad()
        output = model(data)
        target = target.unsqueeze(1)
        target = target.float()
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        acc = (output.round() == target).float().mean()
        epoch_loss.append(loss.item())
        epoch_acc.append(acc)
        progress.set_description(f"Epoch [{epoch}/{epochs}]")
        progress.set_postfix(loss=f"{loss.item():.6f}")
        progress.update(1)


    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in loaders['test']:
            output = model(data)
            target = target.unsqueeze(1)
            target = target.int()
            test_loss += loss_fn(output, target).item()
            pred = output.round()
            correct += pred.eq(target.view_as(pred)).sum()

    test_loss /= len(loaders['test'].dataset)

    train_loss_hist.append(np.mean(epoch_loss))
    train_acc_hist.append(np.mean(epoch_acc))
    test_loss_hist.append(test_loss)
    test_acc_hist.append(correct / len(loaders['test'].dataset))

    if (correct / len(loaders['test'].dataset) >= 0.99) and (epoch % 10 == 0):
        print(f"\nAccuracy on epoch {epoch} = 100%. Finishing training...")
        break

    if epoch > 120:
        if (np.mean(test_acc_hist[-60:]) <= np.mean(test_acc_hist[-120:-60])) or (len(set(test_acc_hist[-50:])) == 1):
            print(f"\nAccuraccy of test set is not improving. Finishing training...")
            break

    print(f" Test set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders['test'].dataset)} ({100. * correct / len(loaders['test'].dataset):.0f}%)\n") if epoch % 5 == 0 else None

print(f"\nAccuracy: {correct}/{len(loaders['test'].dataset)} ({100. * correct / len(loaders['test'].dataset):.0f}%)\n")